In [ ]:
from flask import Flask, render_template, request, redirect, session, jsonify
import spotipy
from spotipy.oauth2 import SpotifyOAuth

app = Flask(__name__)
app.secret_key = '123456'
app.config['SESSION_COOKIE_NAME'] = 'spotify-login-session'

client_id = '162ac8a3eda14927ad6f7e8334dcdf5d'
client_secret = 'c347844247af40e4848a852005a02e73'
redirect_uri = 'http://localhost:5000/callback'


sp_oauth = SpotifyOAuth(client_id=client_id,
                        client_secret=client_secret,
                        redirect_uri=redirect_uri,
                        scope="user-modify-playback-state user-read-playback-state")

@app.route('/')
def index():
    if not session.get('token_info'):
        return redirect('/login')
    return render_template('index.html')

@app.route('/login')
def login():
    auth_url = sp_oauth.get_authorize_url()
    return redirect(auth_url)

@app.route('/callback')
def callback():
    session.clear()
    code = request.args.get('code')
    token_info = sp_oauth.get_access_token(code)
    session['token_info'] = token_info
    return redirect('/')

@app.route('/logout')
def logout():
    session.clear()
    return redirect('/')

@app.route('/get_token')
def get_token():
    if 'token_info' in session:
        return jsonify(session['token_info'])
    return jsonify({'error': 'Not authenticated'})

if __name__ == '__main__':
    app.run(debug=True)




In [ ]:
import streamlit as st
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# URL for Flask app to get token
flask_token_url = 'http://localhost:5000/get_token'

# Fetch access token from Flask
try:
    response = requests.get(flask_token_url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    token_info = response.json()
except requests.exceptions.RequestException as e:
    st.error(f"Error fetching token: {e}")
    st.stop()
except ValueError as e:
    st.error(f"Error decoding JSON: {e}")
    st.stop()

if 'error' in token_info:
    st.error('You are not authenticated. Please log in via the Flask app.')
else:
    sp = spotipy.Spotify(auth=token_info['access_token'])

    st.title("Spotify Song Trimmer")

    # Input for the Spotify track link
    track_link = st.text_input("Enter Spotify Track Link:")

    if track_link:
        # Extract track ID from the link
        track_id = track_link.split("/")[-1].split("?")[0]

        # Fetch track information
        try:
            track_info = sp.track(track_id)
            track_duration_ms = track_info['duration_ms']

            st.write(f"Track: {track_info['name']} by {track_info['artists'][0]['name']}")
            st.write(f"Duration: {track_duration_ms // 1000} seconds")

            # Sliders for trimming the song
            start_time = st.slider(
                "Start Time (seconds)", 0, track_duration_ms // 1000, 0)
            end_time = st.slider(
                "End Time (seconds)", 0, track_duration_ms // 1000, track_duration_ms // 1000)

            if st.button("Play Trimmed Song"):
                # Convert times to milliseconds
                start_ms = start_time * 1000
                end_ms = end_time * 1000

                # Play the song from the start time
                sp.start_playback(uris=[track_info['uri']])
                sp.seek_track(position_ms=start_ms)

                # Wait until the song reaches the end time
                while True:
                    playback = sp.current_playback()
                    current_position = playback['progress_ms']

                    if current_position >= end_ms or not playback['is_playing']:
                        sp.pause_playback()
                        break
                    st.write
                    (f"Current position: {current_position // 1000} seconds")
                    st.sleep(1)
        except spotipy.exceptions.SpotifyException as e:
            st.error(f"Error fetching track info: {e}")


In [ ]:
import streamlit as st
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Spotify credentials
client_id = 'your_client_id'
client_secret = 'your_client_secret'
redirect_uri = 'http://localhost:5000/callback'

# Spotify OAuth object
sp_oauth = SpotifyOAuth(client_id=client_id,
                        client_secret=client_secret,
                        redirect_uri=redirect_uri,
                        scope="user-modify-playback-state user-read-playback-state")

# Fetch access token


def get_token():
    token_info = sp_oauth.get_access_token(as_dict=True)
    return token_info


# Initialize session state
if 'trims' not in st.session_state:
    st.session_state.trims = []
if 'token_info' not in st.session_state:
    st.session_state.token_info = get_token()

sp = spotipy.Spotify(auth=st.session_state.token_info['access_token'])

st.title("Spotify Song Trimmer")

# Input for the Spotify track link
track_link = st.text_input("Enter Spotify Track Link:")

if track_link:
    # Extract track ID from the link
    track_id = track_link.split("/")[-1].split("?")[0]

    # Fetch track information
    track_info = sp.track(track_id)
    track_duration_ms = track_info['duration_ms']

    st.write(f"Track:{track_info['name']} by {track_info['artists'][0]['name']}")

    st.write(f"Duration: {track_duration_ms // 1000} seconds")

    # Slider for trimming the song
    st.write("Use the slider to define multiple trim ranges:")
    trim_ranges = st.slider("Trim Ranges (in seconds)",
                            0, track_duration_ms // 1000,
                            (0, track_duration_ms // 1000),
                            step=1,
                            format="Range: %d - %d")

    # Split the slider value into ranges
    trim_start, trim_end = trim_ranges
    if trim_start < trim_end:
        st.session_state.trims.append({
            'song_uri': track_info['uri'],
            'start_time': trim_start,
            'end_time': trim_end
        })
        st.success("Trim range added successfully!")
    else:
        st.error("Start time must be less than end time.")

    if st.session_state.trims:
        st.write("### Saved Trims")
        for idx, trim in enumerate(st.session_state.trims):
            st.write(f"**Trim {idx + 1}:**")
            st.write(f"Song URI: {trim['song_uri']}")
            st.write(f"Start Time: {trim['start_time']} seconds")
            st.write(f"End Time: {trim['end_time']} seconds")
            if st.button(f"Play Trim {idx + 1}", key=f"play_{idx}"):
                start_ms = trim['start_time'] * 1000
                end_ms = trim['end_time'] * 1000

                # Play the song from the start time
                sp.start_playback(uris=[trim['song_uri']])
                sp.seek_track(position_ms=start_ms)

                # Wait until the song reaches the end time
                while True:
                    playback = sp.current_playback()
                    if playback and playback['is_playing']:
                        current_position = playback['progress_ms']
                        if current_position >= end_ms:
                            sp.pause_playback()
                            break
                        st.write(f"Current position: {current_position // 1000} seconds")

                        st.sleep(1)
                    else:
                        st.write("Playback not active.")
                        break

    if st.button("Clear All Trims"):
        st.session_state.trims = []
        st.success("All trims cleared.")
